# Task
Our task is simple, recognize 10 objects. We will use CIFAR10 dataset for this tutorial.


<a title="CIFAR10" href="https://www.cs.toronto.edu/~kriz/cifar.html"><img width="512" alt="MnistExamples" src="https://production-media.paperswithcode.com/datasets/4fdf2b82-2bc3-4f97-ba51-400322b228b1.png"/></a>

## **Import necessary library**

In [ ]:
import torch
import torchvision
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## **Preprocessing of data**

In [ ]:
## Create dataloader, in PyTorch, we feed the trainer data with use of dataloader
## We create dataloader with dataset from torchvision,
## and we dont have to download it seperately, all automatically done

# Define batch size, batch size is how much data you feed for training in one iteration
batch_size_train = 128
batch_size_test = 1024 #

# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                             ])
#image datasets
train_dataset = torchvision.datasets.CIFAR10('dataset/',
                                           train=True,
                                           download=True,
                                           transform=image_transform)
test_dataset = torchvision.datasets.CIFAR10('dataset/',
                                          train=False,
                                          download=True,
                                          transform=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test,
                                          shuffle=True)

In [ ]:
# We can check the dataloader
_, (example_datas, labels) = next(enumerate(test_loader))
sample = example_datas[0]
# show the data
plt.imshow(sample.permute(1,2,0))
print("Label: "+ str(labels[0]))

## **FCN describtion**

In [ ]:
class FCN(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(32**2*3, 500)
        self.fc2 = nn.Linear(500, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)

        return output

## **CNN describtion**

In [ ]:
## Now we can start to build our CNN model

## Then define the model class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #input channel 1, output channel 10
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5, stride=1)
        #input channel 10, output channel 20
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        #fully connected layer
        self.fc1 = nn.Linear(500, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 500)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x)

## **train and test**

In [ ]:
##define train function
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    train_loss = 0
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        loss.backward()
        optimizer.step()
        counter += 1
        tk0.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))
    train_loss /= len(train_loader.dataset)
    return train_loss
##define test function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

## **Choice of optimizer and device**

In [ ]:
## create model and optimizer
learning_rate = 0.01
momentum = 0.5
device = "cpu"
model = CNN().to(device) #using cpu here
optimizer = optim.Adam(model.parameters())

## **Learning process**

In [ ]:
num_epoch = 15
train_loss_epochs = []
test_loss_epochs = []
for epoch in range(1, num_epoch + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch)
        train_loss_epochs.append(train_loss)
        test_loss = test(model, device, test_loader)
        test_loss_epochs.append(test_loss)
plt.plot(train_loss_epochs)
plt.plot(test_loss_epochs)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

##**Compare results**

In [ ]:
from torchsummary import summary
summary(model, (1, 28, 28))

In [ ]:
model